In [1]:
import psycopg2 as pg
import pandas.io.sql as psql

In [2]:
import datetime

In [3]:
import pandas as pd
import numpy as np

In [4]:
start = datetime.date(2016,11,1)
end = datetime.date(2016,11,14)

In [5]:
s = str(start)
e = str(end)

In [6]:
print(s,e)

2016-11-01 2016-11-14


In [2]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    
    #処方ビュー
    view_os = """
       SELECT v.pt_id
              ,v.pt_name
              ,v.bunsho_date AS start_date
              ,v.steroid_a_day
              ,v.yoho
              ,v.steroid_amount AS s_sum
              ,v.koshin_kacd
         FROM v_steroid_amount2 AS v
        WHERE (v.steroid_a_day >= 20 AND v.yoho >= 14 OR v.steroid_a_day IS NULL AND v.yoho IS NOT NULL)
          AND v.bunsho_date BETWEEN '{0}' AND '{1}'
        ORDER BY v.pt_id;""".format(s,e)
    
    #注射ビュー（ステロイド以外）
    view_esi = """
       SELECT v.pt_id
              ,v.pt_name
              ,MIN(v.bunsho_date) AS start_date
              ,MAX(v.bunsho_date) AS last_date
              ,MAX(v.bunsho_date) - (MIN(v.bunsho_date) - 1) AS injection_period
              ,v.yoho
              ,SUM(v.steroid_a_day) AS steroid_a_day
              ,SUM(v.steroid_amount) AS s_sum
              ,MAX(v.koshin_kacd) AS koshin_kacd
         FROM v_steroid_amount2 AS v
        WHERE (v.yoho IS NULL AND v.steroid_a_day IS NULL)
          AND v.bunsho_date BETWEEN '{0}' AND '{1}'
     GROUP BY v.pt_id,v.pt_name,v.yoho
     ORDER BY v.pt_id;""".format(s,e)
    
    #注射ビュー（ステロイド）
    view_si = """
       SELECT v.pt_id
              ,v.pt_name
              ,MIN(v.bunsho_date) AS start_date
              ,MAX(v.bunsho_date) AS last_date
              ,MAX(v.bunsho_date) - (MIN(v.bunsho_date) - 1) AS injection_period
              ,v.yoho
              ,SUM(v.steroid_a_day) AS steroid_a_day
              ,SUM(v.steroid_amount) AS s_sum
              ,MAX(v.koshin_kacd) AS koshin_kacd
         FROM v_steroid_amount2 AS v
        WHERE (v.yoho IS NULL AND v.steroid_a_day IS NOT NULL)
          AND v.bunsho_date BETWEEN '{0}' AND '{1}'
     GROUP BY v.pt_id,v.pt_name,v.yoho
     ORDER BY v.pt_id;""".format(s,e)
    
    kenreki = """SELECT *
                   FROM kenreki
                  WHERE 文書日付 <= '%s'
               ORDER BY 患者id,検査コード_x,文書日付;"""%e
    
    # Added '身長cm', '体重kg' and 'bmi割合' on Aug 30, 2016
    kekka = "SELECT 患者id,death_f,身長cm,体重kg,bmi割合 FROM kekka ORDER BY 患者id;"
    
    baraclude = "SELECT * FROM baraclude;"
    
    yoyaku = "SELECT * FROM yoyaku;"
    
    kamaster = "SELECT * FROM kamaster;"
    
    os = psql.read_sql(view_os,conn)
    esi = psql.read_sql(view_esi,conn)
    si = psql.read_sql(view_si,conn)
    kn = psql.read_sql(kenreki,conn)
    ke = psql.read_sql(kekka,conn)
    br = psql.read_sql(baraclude,conn)
    yo = psql.read_sql(yoyaku,conn)
    ka = psql.read_sql(kamaster,conn)
print(os,esi,si,kn,ke,br,yo,ka)

'\nwith pg.connect(database=\'uonuma_kikan\',user=\'postgres\',host=\'localhost\',port=5432,password=12345) as conn:\n    \n    #処方ビュー\n    view_os = """\n       SELECT v.pt_id\n              ,v.pt_name\n              ,v.bunsho_date AS start_date\n              ,v.steroid_a_day\n              ,v.yoho\n              ,v.steroid_amount AS s_sum\n              ,v.koshin_kacd\n         FROM v_steroid_amount2 AS v\n        WHERE (v.steroid_a_day >= 20 AND v.yoho >= 14 OR v.steroid_a_day IS NULL AND v.yoho IS NOT NULL)\n          AND v.bunsho_date BETWEEN \'{0}\' AND \'{1}\'\n        ORDER BY v.pt_id;""".format(s,e)\n    \n    #注射ビュー（ステロイド以外）\n    view_esi = """\n       SELECT v.pt_id\n              ,v.pt_name\n              ,MIN(v.bunsho_date) AS start_date\n              ,MAX(v.bunsho_date) AS last_date\n              ,MAX(v.bunsho_date) - (MIN(v.bunsho_date) - 1) AS injection_period\n              ,v.yoho\n              ,SUM(v.steroid_a_day) AS steroid_a_day\n              ,SUM(v.steroid_a

In [3]:
br

In [9]:
br['用法の日数回数'] = br['用法の日数回数'].apply(lambda x:datetime.timedelta(x))

In [10]:
br['文書日付'] = pd.to_datetime(br['文書日付'])

In [11]:
br['最終投与日'] = br['文書日付'] + br['用法の日数回数'] - datetime.timedelta(1)

In [4]:
br

In [5]:
br.sort_values('患者id')

In [14]:
br = br.groupby('患者id').agg({'最終投与日':'max'})

In [15]:
br.reset_index(inplace=True)

In [16]:
br.head()

,患者id,最終投与日
0,108154,2016-01-28
1,140317,2015-11-10
2,167835,2016-11-16
3,181161,2017-02-01
4,181285,2016-04-17


In [17]:
os.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 7 columns):
pt_id            63 non-null object
pt_name          63 non-null object
start_date       63 non-null object
steroid_a_day    20 non-null float64
yoho             63 non-null float64
s_sum            20 non-null float64
koshin_kacd      63 non-null object
dtypes: float64(3), object(4)
memory usage: 2.5+ KB


In [18]:
os['start_date'] = pd.to_datetime(os['start_date'])

In [6]:
os.head()

In [20]:
os.shape

(63, 7)

In [21]:
esi['start_date'] = pd.to_datetime(esi['start_date'])

In [22]:
esi.head()

,pt_id,pt_name,start_date,last_date,injection_period,yoho,steroid_a_day,s_sum,koshin_kacd


In [23]:
esi.shape

(0, 9)

In [24]:
si['start_date'] = pd.to_datetime(si['start_date'])

In [7]:
si

In [26]:
si.shape

(38, 9)

In [27]:
osb = pd.merge(os,br,left_on='pt_id',right_on='患者id',how='left')
esib = pd.merge(esi,br,left_on='pt_id',right_on='患者id',how='left')
sib = pd.merge(si,br,left_on='pt_id',right_on='患者id',how='left')

In [8]:
osb.head()

In [29]:
osb.shape

(63, 9)

In [30]:
esib.head()

,pt_id,pt_name,start_date,last_date,injection_period,yoho,steroid_a_day,s_sum,koshin_kacd,患者id,最終投与日


In [31]:
esib.shape

(0, 11)

In [9]:
sib

In [33]:
sib.shape

(38, 11)

In [10]:
osb = osb[(osb['start_date'] > osb['最終投与日'])|(osb['最終投与日'].isnull())]
osb

In [35]:
esib = esib[(esib['start_date'] > esib['最終投与日'])|(esib['最終投与日'].isnull())]
esib

,pt_id,pt_name,start_date,last_date,injection_period,yoho,steroid_a_day,s_sum,koshin_kacd,患者id,最終投与日


In [11]:
sib = sib[(sib['start_date'] > sib['最終投与日'])|(sib['最終投与日'].isnull())]
sib

In [37]:
print(osb.shape,esib.shape,sib.shape)

(63, 9) (0, 11) (38, 11)


In [38]:
# '身長cm', '体重kg', 'bmi割合' will be appeared
ke.head()

,患者id,death_f,身長cm,体重kg,bmi割合
0,100005,False,152.1,53.0,22.909596
1,100013,None,88.9,13.2,16.702074
2,100021,False,146.8,46.3,21.484679
3,100030,None,163.2,48.5,18.209642
4,100048,None,NaN,NaN,NaN


In [39]:
osbk = pd.merge(osb,ke,left_on='pt_id',right_on='患者id',how='left')
esibk = pd.merge(esib,ke,left_on='pt_id',right_on='患者id',how='left')
sibk = pd.merge(sib,ke,left_on='pt_id',right_on='患者id',how='left')

In [40]:
print(osbk.shape,esibk.shape,sibk.shape)

(63, 14) (0, 16) (38, 16)


In [41]:
# Remove dead patients
osbk = osbk[osbk['death_f']!=True]
esibk = esibk[esibk['death_f']!=True]
sibk = sibk[sibk['death_f']!=True]

In [42]:
print(osbk.shape,esibk.shape,sibk.shape)

(63, 14) (0, 16) (36, 16)


In [12]:
kn

In [44]:
kn.columns

Index(['index', '患者id', '文書日付_d', '伝票名称', '検査コード_x', '検査名称_x', '結果_x',
       '結果（数値）', 'key', '検査コード_y', '検査名称_y', '結果_y', '文書日付'],
      dtype='object')

In [45]:
knm = kn[['患者id','文書日付','検査コード_x','検査名称_x','結果（数値）','結果_y']]

In [13]:
knm[knm['結果_y'].isnull()]

In [47]:
knm['結果_y'] = knm['結果_y'].fillna('ｹﾝｻﾁｭｳ')

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [48]:
knm[knm['結果_y'].isnull()]

,患者id,文書日付,検査コード_x,検査名称_x,結果（数値）,結果_y


In [14]:
knm[knm['結果_y']=='ｹﾝｻﾁｭｳ']

In [50]:
#knm.to_excel('knm.xlsx')

In [51]:
# added '2011700' on 16 Nov, 2016
# '2011700' = ＨＢＶＤＮＡ（新）定量
# '2011701' = ＨＢＶＤＮＡ（新）定性
cd = ['2011600','2011700','2012000']
# cd: ＨＢＶＤＮＡ関連検査の定量検査コード

In [52]:
set(knm[knm['検査コード_x'].isin(cd)]['結果_y'].tolist())

{'(-)',
 '2.1',
 '2.2',
 '2.3',
 '2.4',
 '2.5',
 '2.6',
 '2.7',
 '2.8',
 '2.9',
 '3',
 '3.1',
 '3.2',
 '3.3',
 '3.4',
 '3.5',
 '3.6',
 '3.7',
 '3.8',
 '4',
 '4.1',
 '4.2',
 '4.7',
 '4.8',
 '5.1',
 '5.9',
 '7.6',
 '8.5',
 '<2.1+',
 'ｹﾝｻﾁｭｳ',
 'ﾐｹﾝｼﾕﾂ'}

In [53]:
# 検査コード_x:定量検査コード
#　結果_y:定性結果
# 結果_x:定量結果
'''
+----------+ +----------+
|検歴（定量）x | |検歴（定性）y|
|          | |          |
+----------+ +----------+

teisei = [2010100,2010200,2011400,2012001,2010702,2010802,2011601,2011701]
teiryo = [2010101,2010201,2011401,2012000,2010701,2010801,2011600,2011700]
# 2011600 has both sides of character, qualitative and quantitative.
# On 16 Nov, 2016 we added '2011701' in teisei and '2011700' in teiryo.
...
rk1 = rk[rk['検査コード'].isin(teisei)]
rk2 = rk[rk['検査コード'].isin(teiryo)]
...
rkc = pd.merge(rk2_,rk1_,on='key',how='left')

"cited from HBV_import"

'''
def xchange(df):
    for x in df.index:
        if df.loc[x,'検査コード_x'] in cd:
            if df.loc[x,'結果_y']=='ﾐｹﾝｼﾕﾂ':
                df.loc[x,'結果_y']='(-)'
            elif df.loc[x,'結果_y']=='ｹﾝｻﾁｭｳ':
                pass
            elif df.loc[x,'結果_y']=='(-)':
                pass
            else:
                df.loc[x,'結果_y']='(+)'
        else:
            pass

In [54]:
xchange(knm)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [55]:
#knm.to_excel('knm_xchange_%s.xlsx'%e)

In [56]:
set(knm[knm['検査コード_x'].isin(cd)]['結果_y'].tolist())

{'(+)', '(-)', 'ｹﾝｻﾁｭｳ'}

In [57]:
print(osbk.shape,esibk.shape,sibk.shape)

(63, 14) (0, 16) (36, 16)


In [58]:
osbkk = pd.merge(osbk,knm,left_on='pt_id',right_on='患者id',how='left')
esibkk = pd.merge(esibk,knm,left_on='pt_id',right_on='患者id',how='left')
sibkk = pd.merge(sibk,knm,left_on='pt_id',right_on='患者id',how='left')

In [59]:
print(osbkk.shape,esibkk.shape,sibkk.shape)

(220, 20) (0, 22) (68, 22)


In [15]:
yo.head()

In [61]:
yo['診療科コード'][0]

' 41'

In [62]:
osbkk['koshin_kacd'][0]

'20'

In [63]:
def xreplace(x):
    if x[0] == '0':
        return x[1:]
    else:
        return x

In [64]:
yo['診療科コード'] = yo['診療科コード'].apply(lambda x:x.replace(' ','')).apply(xreplace)

In [65]:
yo['診療科コード'][0]

'41'

In [66]:
# *******************(以下作業中・・作業完了2016.10.17)***********************

In [67]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    
    #ビュー
    view = """
       SELECT *
         FROM v_steroid_amount2
        WHERE bunsho_date BETWEEN '{0}' AND '{1}'
        ORDER BY pt_id;""".format(s,e)
    vw = psql.read_sql(view,conn)

In [68]:
import numpy as np

In [69]:
yo['key'] = yo['患者id']+yo['診療科コード']

In [70]:
def xkey(dfx):
    try:
        dfx['key'] = dfx['pt_id']+dfx['koshin_kacd']
    except:
        pass

In [71]:
xkey(osbkk)
xkey(esibkk)
xkey(sibkk)

In [16]:
sibkk

In [73]:
def xmerge(dfx,dfy):
    try:
        dfx = pd.merge(dfx,dfy,on='key',how='left')
        dfx['予約日時'] = ''
        dfx['診療科コード'] = ''
        dfx['診療科名'] = ''
    except:
        pass

In [18]:
osbkky = pd.merge(osbkk,yo,on='key',how='left')
osbkky['予約日時'] = ''
osbkky['診療科コード'] = ''
osbkky['診療科名'] = ''
osbkky

"\nosbkky = pd.merge(osbkk,yo,on='key',how='left')\nosbkky['予約日時'] = ''\nosbkky['診療科コード'] = ''\nosbkky['診療科名'] = ''\nosbkky\n"

In [75]:
esibkky = pd.merge(esibkk,yo,on='key',how='left')
esibkky['予約日時'] = ''
esibkky['診療科コード'] = ''
esibkky['診療科名'] = ''
esibkky

,pt_id,pt_name,start_date,last_date,injection_period,yoho,steroid_a_day,s_sum,koshin_kacd,患者id_x,...,予約日時,予約時刻,グループコード,グループsfx,予約枠名称,診療科医師コード,診療科医師名,診療科コード,診療科名,key


In [19]:
sibkky = pd.merge(sibkk,yo,on='key',how='left')
sibkky['予約日時'] = ''
sibkky['診療科コード'] = ''
sibkky['診療科名'] = ''
sibkky

"\nsibkky = pd.merge(sibkk,yo,on='key',how='left')\nsibkky['予約日時'] = ''\nsibkky['診療科コード'] = ''\nsibkky['診療科名'] = ''\nsibkky\n"

In [77]:
vw['key'] = vw['pt_id']+vw['koshin_kacd']
vw['key'] = vw['key'].fillna('0000000000')
zkeys = np.unique(vw['key']).tolist()

In [78]:
zkeys

['0000000000',
 '1038296',
 '10816254',
 '10970320',
 '1099596',
 '11017511',
 '1114229',
 '1121196',
 '1123999',
 '1130511',
 '12115116',
 '12704316',
 '12783311',
 '1314409',
 '13167920',
 '13467831',
 '1357952',
 '13790120',
 '13825816',
 '15413056',
 '15684150',
 '1594339',
 '16044031',
 '16330911',
 '1637341',
 '16846728',
 '16906452',
 '17017850',
 '17111531',
 '17956652',
 '181639',
 '183681',
 '18373320',
 '18378452',
 '18790956',
 '18933228',
 '19179541',
 '1970450',
 '20267311',
 '2055759',
 '2092871',
 '20928711',
 '21045528',
 '21085411',
 '21094341',
 '21353552',
 '2140741',
 '21679811',
 '21876611',
 '21930450',
 '22061211',
 '2209226',
 '22219416',
 '2239011',
 '2258641',
 '2268409',
 '22903216',
 '22945828',
 '23452447',
 '24029011',
 '2417331',
 '24983131',
 '25191731',
 '25396152',
 '2544526',
 '2546066',
 '2596679',
 '25991811',
 '26180756',
 '2630369',
 '26318411',
 '26420220',
 '2661086',
 '26730947',
 '26790258',
 '26821616',
 '27115231',
 '27206020',
 '27447052',

In [79]:
# dfx = osbkky,esibkky,sibkky
# dfy = yo
def xyoyaku(dfx,dfy):
    if len(dfx) > 0:
        for k in zkeys:
            dfy_k = dfy[dfy['key']==k]
            k_min = dfy_k[dfy_k['予約日時']==dfy_k['予約日時'].min()].drop_duplicates('予約日時')
            for i in dfx.index:
                if dfx.loc[i,'key'] == k:
                    try:
                        dfx.loc[i,'予約日時'] = k_min['予約日時'].iloc[0]
                        dfx.loc[i,'診療科コード'] = k_min['診療科コード'].iloc[0]
                        dfx.loc[i,'診療科名'] = k_min['診療科名'].iloc[0]
                    except:
                        dfx.loc[i,'予約日時'] = ''
                        dfx.loc[i,'診療科コード'] = ''
                        dfx.loc[i,'診療科名'] = ''
                elif dfx.loc[i,'key'] != k:
                    if dfx.loc[i,'pt_id'] == k[-2:]:
                        try:
                            dfx.loc[i,'予約日時'] = k_min['予約日時'].iloc[0]
                            dfx.loc[i,'診療科コード'] = k_min['診療科コード'].iloc[0]
                            dfx.loc[i,'診療科名'] = k_min['診療科名'].iloc[0]
                        except:
                            dfx.loc[i,'予約日時'] = ''
                            dfx.loc[i,'診療科コード'] = ''
                            dfx.loc[i,'診療科名'] = ''
                    else:
                        pass
                else:
                    pass
    elif len(dfx) == 0:
        pass
    else:
        pass

In [80]:
xyoyaku(osbkky,yo)

In [20]:
osbkky

In [82]:
xyoyaku(esibkky,yo)

In [83]:
esibkky

,pt_id,pt_name,start_date,last_date,injection_period,yoho,steroid_a_day,s_sum,koshin_kacd,患者id_x,...,予約日時,予約時刻,グループコード,グループsfx,予約枠名称,診療科医師コード,診療科医師名,診療科コード,診療科名,key


In [84]:
xyoyaku(sibkky,yo)

In [21]:
sibkky

In [86]:
#osbkky.to_excel('func_test_osbkky_201610171222.xlsx')
#esibkky.to_excel('func_test_esibkky_201610171222.xlsx')
#sibkky.to_excel('func_test_sibkky_201610171222.xlsx')

In [87]:
# *******************(以上作業中・・作業完了2016.10.17)***********************

In [88]:
#osbkky = pd.merge(osbkk,yo,left_on=['pt_id','koshin_kacd'],right_on=['患者id','診療科コード'],how='left')
#esibkky = pd.merge(esibkk,yo,left_on=['pt_id','koshin_kacd'],right_on=['患者id','診療科コード'],how='left')
#sibkky = pd.merge(sibkk,yo,left_on=['pt_id','koshin_kacd'],right_on=['患者id','診療科コード'],how='left')

In [89]:
#osbkky.to_excel('osbkky.xlsx')

In [90]:
#esibkky.to_excel('esibkky.xlsx')

In [91]:
#sibkky.to_excel('sibkky.xlsx')

In [92]:
print(osbkky.shape,esibkky.shape,sibkky.shape)

(220, 33) (0, 35) (68, 35)


In [22]:
osbkky.head(2)

In [94]:
ka = ka[['kacd','kaname']]

In [95]:
osbkkyk = pd.merge(osbkky,ka,left_on='koshin_kacd',right_on='kacd',how='left')
esibkkyk = pd.merge(esibkky,ka,left_on='koshin_kacd',right_on='kacd',how='left')
sibkkyk = pd.merge(sibkky,ka,left_on='koshin_kacd',right_on='kacd',how='left')

In [96]:
print(osbkkyk.shape,esibkkyk.shape,sibkkyk.shape)

(220, 35) (0, 37) (68, 37)


In [97]:
# Qery 0

# No conditions

osbkkyk.to_excel('qOS0_mod_%s.xlsx'%e)
esibkkyk.to_excel('qESI0_mod_%s.xlsx'%e)
sibkkyk.to_excel('qSI0_mod_%s.xlsx'%e)

In [98]:
# Qery 1

# hbs抗原_検査日 is null

list1a = osbkkyk[osbkkyk['検査コード_x']=='2010101']['pt_id'].tolist()
osbkkyk[~osbkkyk['pt_id'].isin(list1a)].to_excel('qOS1_mod_%s.xlsx'%e)

list1b = esibkkyk[esibkkyk['検査コード_x']=='2010101']['pt_id'].tolist()
esibkkyk[~esibkkyk['pt_id'].isin(list1b)].to_excel('qESI1_mod_%s.xlsx'%e)

list1c = sibkkyk[sibkkyk['検査コード_x']=='2010101']['pt_id'].tolist()
sibkkyk[~sibkkyk['pt_id'].isin(list1c)].to_excel('qSI1_mod_%s.xlsx'%e)

In [99]:
# Query 2

# hbs抗原_結果 = '(+)'
# OR 
# hbv_dna_結果 = '(+)'

list2a = osbkkyk[\
(((osbkkyk['検査コード_x']=='2010101')&(osbkkyk['結果_y']=='(+)'))|\
 ((osbkkyk['検査コード_x']=='2010101')&(osbkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))|\
(((osbkkyk['検査コード_x'].isin(cd))&(osbkkyk['結果_y']=='(+)'))|\
 ((osbkkyk['検査コード_x'].isin(cd))&(osbkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))]['pt_id'].tolist()
osbkkyk[osbkkyk['pt_id'].isin(list2a)].to_excel('qOS2_mod_%s.xlsx'%e)

list2b = esibkkyk[\
(((esibkkyk['検査コード_x']=='2010101')&(esibkkyk['結果_y']=='(+)'))|\
 ((esibkkyk['検査コード_x']=='2010101')&(esibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))|\
(((esibkkyk['検査コード_x'].isin(cd))&(esibkkyk['結果_y']=='(+)'))|\
 ((esibkkyk['検査コード_x'].isin(cd))&(esibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))]['pt_id'].tolist()
esibkkyk[esibkkyk['pt_id'].isin(list2b)].to_excel('qESI2_mod_%s.xlsx'%e)

list2c = sibkkyk[\
(((sibkkyk['検査コード_x']=='2010101')&(sibkkyk['結果_y']=='(+)'))|\
 ((sibkkyk['検査コード_x']=='2010101')&(sibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))|\
(((sibkkyk['検査コード_x'].isin(cd))&(sibkkyk['結果_y']=='(+)'))|\
 ((sibkkyk['検査コード_x'].isin(cd))&(sibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))]['pt_id'].tolist()
sibkkyk[sibkkyk['pt_id'].isin(list2c)].to_excel('qSI2_mod_%s.xlsx'%e)

In [100]:
# Query 3-1

# (hbc抗体_結果 = '(+)' OR hbs抗体_結果 = '(+)')
# AND 
# hbv_dna_検査日 IS NULL

list31ax = osbkkyk[\
(((osbkkyk['検査コード_x']=='2011401')&(osbkkyk['結果_y']=='(+)'))|\
 ((osbkkyk['検査コード_x']=='2011401')&(osbkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))|\
(((osbkkyk['検査コード_x']=='2010201')&(osbkkyk['結果_y']=='(+)'))|\
 ((osbkkyk['検査コード_x']=='2010201')&(osbkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))]['pt_id'].tolist()
list31ay = osbkkyk[osbkkyk['検査コード_x'].isin(cd)]['pt_id'].tolist()
tmp31a = osbkkyk[osbkkyk['pt_id'].isin(list31ax)]
tmp31a[~tmp31a['pt_id'].isin(list31ay)].to_excel('qOS31_mod_%s.xlsx'%e)

list31bx = esibkkyk[\
(((esibkkyk['検査コード_x']=='2011401')&(esibkkyk['結果_y']=='(+)'))|\
 ((esibkkyk['検査コード_x']=='2011401')&(esibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))|\
(((esibkkyk['検査コード_x']=='2010201')&(esibkkyk['結果_y']=='(+)'))|\
 ((esibkkyk['検査コード_x']=='2010201')&(esibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))]['pt_id'].tolist()
list31by = esibkkyk[esibkkyk['検査コード_x'].isin(cd)]['pt_id'].tolist()
tmp31b = esibkkyk[esibkkyk['pt_id'].isin(list31bx)]
tmp31b[~tmp31b['pt_id'].isin(list31by)].to_excel('qESI31_mod_%s.xlsx'%e)

list31cx = sibkkyk[\
(((sibkkyk['検査コード_x']=='2011401')&(sibkkyk['結果_y']=='(+)'))|\
 ((sibkkyk['検査コード_x']=='2011401')&(sibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))|\
(((sibkkyk['検査コード_x']=='2010201')&(sibkkyk['結果_y']=='(+)'))|\
 ((sibkkyk['検査コード_x']=='2010201')&(sibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))]['pt_id'].tolist()
list31cy = sibkkyk[sibkkyk['検査コード_x'].isin(cd)]['pt_id'].tolist()
tmp31c = sibkkyk[sibkkyk['pt_id'].isin(list31cx)]
tmp31c[~tmp31c['pt_id'].isin(list31cy)].to_excel('qSI31_mod_%s.xlsx'%e)

In [101]:
# Query 3-2

# (hbc抗体_結果 = '(+)' OR hbs抗体_結果 = '(+)') 
# AND
# hbv_dna_結果 = '(-)'

list32ax = osbkkyk[\
(((osbkkyk['検査コード_x']=='2011401')&(osbkkyk['結果_y']=='(+)'))|\
 ((osbkkyk['検査コード_x']=='2011401')&(osbkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))|\
(((osbkkyk['検査コード_x']=='2010201')&(osbkkyk['結果_y']=='(+)'))|\
 ((osbkkyk['検査コード_x']=='2010201')&(osbkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))]['pt_id'].tolist()
list32ay = osbkkyk[(osbkkyk['検査コード_x'].isin(cd))&(osbkkyk['結果_y']=='(-)')]['pt_id'].tolist()
tmp32a = osbkkyk[osbkkyk['pt_id'].isin(list32ax)]
tmp32a[tmp32a['pt_id'].isin(list32ay)].to_excel('qOS32_mod_%s.xlsx'%e)

list32bx = esibkkyk[\
(((esibkkyk['検査コード_x']=='2011401')&(esibkkyk['結果_y']=='(+)'))|\
 ((esibkkyk['検査コード_x']=='2011401')&(esibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))|\
(((esibkkyk['検査コード_x']=='2010201')&(esibkkyk['結果_y']=='(+)'))|\
 ((esibkkyk['検査コード_x']=='2010201')&(esibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))]['pt_id'].tolist()
list32by = esibkkyk[(esibkkyk['検査コード_x'].isin(cd))&(esibkkyk['結果_y']=='(-)')]['pt_id'].tolist()
tmp32b = esibkkyk[esibkkyk['pt_id'].isin(list32bx)]
tmp32b[tmp32b['pt_id'].isin(list32by)].to_excel('qESI32_mod_%s.xlsx'%e)

list32cx = sibkkyk[\
(((sibkkyk['検査コード_x']=='2011401')&(sibkkyk['結果_y']=='(+)'))|\
 ((sibkkyk['検査コード_x']=='2011401')&(sibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))|\
(((sibkkyk['検査コード_x']=='2010201')&(sibkkyk['結果_y']=='(+)'))|\
 ((sibkkyk['検査コード_x']=='2010201')&(sibkkyk['結果_y']=='ｹﾝｻﾁｭｳ')))]['pt_id'].tolist()
list32cy = sibkkyk[(sibkkyk['検査コード_x'].isin(cd))&(sibkkyk['結果_y']=='(-)')]['pt_id'].tolist()
tmp32c = sibkkyk[sibkkyk['pt_id'].isin(list32cx)]
tmp32c[tmp32c['pt_id'].isin(list32cy)].to_excel('qSI32_mod_%s.xlsx'%e)

In [102]:
# Query 4

# hbs抗原_結果 = '(-)' 
# AND 
# (hbc抗体_検査日 IS NULL OR hbs抗体_検査日 IS NULL)

list4ax = osbkkyk[(osbkkyk['検査コード_x']=='2010101')&(osbkkyk['結果_y']=='(-)')]['pt_id'].tolist()
list4ay = osbkkyk[(osbkkyk['検査コード_x']=='2011401')|(osbkkyk['検査コード_x']=='2010201')]['pt_id'].tolist()
tmp4a = osbkkyk[osbkkyk['pt_id'].isin(list4ax)]
tmp4a[~tmp4a['pt_id'].isin(list4ay)].to_excel('qOS4_mod_%s.xlsx'%e)

list4bx = esibkkyk[(esibkkyk['検査コード_x']=='2010101')&(esibkkyk['結果_y']=='(-)')]['pt_id'].tolist()
list4by = esibkkyk[(esibkkyk['検査コード_x']=='2011401')|(esibkkyk['検査コード_x']=='2010201')]['pt_id'].tolist()
tmp4b = esibkkyk[esibkkyk['pt_id'].isin(list4bx)]
tmp4b[~tmp4b['pt_id'].isin(list4by)].to_excel('qESI4_mod_%s.xlsx'%e)

list4cx = sibkkyk[(sibkkyk['検査コード_x']=='2010101')&(sibkkyk['結果_y']=='(-)')]['pt_id'].tolist()
list4cy = sibkkyk[(sibkkyk['検査コード_x']=='2011401')|(sibkkyk['検査コード_x']=='2010201')]['pt_id'].tolist()
tmp4c = sibkkyk[sibkkyk['pt_id'].isin(list4cx)]
tmp4c[~tmp4c['pt_id'].isin(list4cy)].to_excel('qSI4_mod_%s.xlsx'%e)